# CityBikes

Send a request to CityBikes for the city of your choice. 

In [51]:
# importing required libraries
import requests
import json
import pandas as pd

# function to fetch networks for a city
'''
def city_bikes_per_city(city_name):
    # request to fetch data
    response = requests.get('http://api.citybik.es/v2/networks')
    
    # checking to see if request was successful
    if response.status_code == 200:
        data = response.json()
        
        
        # list to hold networks found in city
        networks_in_city = []
        
        # filtering networks by city
        for network in data['networks']:
            if network['location']['city'].lower() == city_name.lower(): # making sure to make it insensitive to case
                networks_in_city.append(network)
                
        # if there are networks in city
        if networks_in_city:
            for network in networks_in_city:
                print(json.dumps(network, indent=4))
                
                # loading station data for each network
                network_id = network['id']
                station_reponse = requests.get(f'http://api.citybik.es/v2/networks/{network_id}')
                
                if station_response.status_code == 200:
                    station_data = station_response.json()
                    
                    # looking for the first station in the list
                    if 'network' in station_data and station_data['network']['stations']:
                        first_station = station_data['network']['stations'][0]
                        print(json.dumps(first_station, indent=4))
target_city = 'Paris'
city_bikes_per_city(target_city)
'''

{
    "id": "velib",
    "name": "V\u00e9lib' M\u00e9tropole",
    "location": {
        "latitude": 48.856614,
        "longitude": 2.3522219,
        "city": "Paris",
        "country": "FR"
    },
    "href": "/v2/networks/velib",
    "company": [
        "Smovengo"
    ],
    "license": {
        "name": "OPEN LICENCE 2.0",
        "url": "etalab.gouv.fr/wp-content/uploads/2018/11/open-licence.pdf"
    },
    "ebikes": true,
    "gbfs_href": "https://velib-metropole-opendata.smovengo.cloud/opendata/Velib_Metropole/gbfs.json"
}
{
    "id": "004f9aea9391c0a3197981c85926fdb1",
    "name": "AUH - Marasy",
    "latitude": 24.451202,
    "longitude": 54.33451,
    "timestamp": "2024-10-21T05:18:22.733265Z",
    "free_bikes": 2,
    "empty_slots": 11,
    "extra": {
        "uid": "113",
        "renting": true,
        "returning": true,
        "last_updated": 1729487739,
        "address": "AUH AL BATEEN",
        "post_code": "AUH",
        "payment": [
            "key",
            

Now that we know a single request works, lets create a loop.

In [52]:
# now that we know the structure of the api, we can edit this function to output a dataframe of the necessary data
def city_bikes_per_city(city_name):
    '''
    This function gets bike station data for a city and returns the data as a dataframe.
    
    Parameters:
    city_name (string): the name of the city to fetch bike station data for
    
    Returns:
    dataframe: a dataframe containing station information.
    '''
    # request to fetch data
    response = requests.get('http://api.citybik.es/v2/networks')
    
    # checking to see if request was successful
    if response.status_code == 200:
        data = response.json()
        
        
        # list to hold networks found in city
        networks_in_city = []
        
        # filtering networks by city
        for network in data['networks']:
            if network['location']['city'].lower() == city_name.lower(): # making sure to make it insensitive to case
                networks_in_city.append(network)
        
        # list to hold necessary station data
        station_network_data = []
        
        # if there are networks in city
        if networks_in_city:
            for network in networks_in_city:
                # loading station data for each network
                network_id = network['id']
                station_response = requests.get(f'http://api.citybik.es/v2/networks/{network_id}')
                
                if station_response.status_code == 200:
                    station_data = station_response.json()
                    
                    # extracting station info
                    if 'network' in station_data and station_data['network']['stations']:
                        for station in station_data['network']['stations']:
                            station_info = {
                                'city': network['location']['city'],
                                'country': network['location']['country'],
                                'longitude': station['longitude'],
                                'latitude': station['latitude'],
                                'free_bikes': station['free_bikes'],
                                'empty_slots': station['empty_slots']
                            }
                            station_network_data.append(station_info)
                else:
                    print(f"Failed to fetch station data for {network['name']}: {station_response.status_code} - {station_response.text}")
        # creating dataframe 
        stations_df = pd.DataFrame(station_network_data)
        return stations_df 
    else:
        print('Failed to fetch networks.')
        return None
                    
                    
target_city = 'Barcelona'
city_bikes_per_city(target_city)

,city,country,longitude,latitude,free_bikes,empty_slots
0,Barcelona,ES,2.162255,41.405520,3,11
1,Barcelona,ES,2.182508,41.396717,23,10
2,Barcelona,ES,2.169427,41.386543,19,8
3,Barcelona,ES,2.150807,41.384810,21,3
4,Barcelona,ES,2.204141,41.399217,25,0
...,...,...,...,...,...,...
509,Barcelona,ES,2.144083,41.398573,9,15
510,Barcelona,ES,2.187813,41.370248,1,27
511,Barcelona,ES,2.216209,41.405365,17,10
512,Barcelona,ES,2.152180,41.385400,10,15


In [55]:
city_bikes_per_city(target_city).min()

city           Barcelona
country               ES
longitude       2.109154
latitude       41.346775
free_bikes             0
empty_slots            0
dtype: object

Put your parsed results into a DataFrame.

In [56]:
city_bikes_per_city(target_city).max()

city           Barcelona
country               ES
longitude       2.220691
latitude       41.462095
free_bikes            48
empty_slots           50
dtype: object

In [57]:
city_bikes_per_city(target_city).to_csv('city_bikes_per_city_barcelona', index=False)